In [13]:
!pip install bs_ds
!pip install fake_useragent
!pip install lxml
!pip install BeautifulSoup4

In [142]:
#Tipología y ciclo de vida de los datos
#Práctica 1
#Autor: Eduardo Bejar

#Importar lo necesario
import requests
import random
from bs4 import BeautifulSoup
import csv

#Función para remover etiquetas html
#Fuente: https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

#Guardar encabezado en archivo CSV
with open('montanasSummitPost.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["No.", "Nombre_montana", "Altura", "Score", "Vistas", "Ubicacion"])


#Fuente de datos elegida: SummitPost - https://www.summitpost.org/
#Sitio de información sobre elevaciones y montañas que existen en el mundo y unos pocos casos de fuera de la Tierra

#Contiene en total 629 páginas y 15.086 elevaciones que se consultan por separado 
#en páginas de hasta 24 elementos, a través de parámetro en la URL

#Número inicial de variable de recorrido de páginas
j = 0

#Para sumar las montañas
k = 1

#Loop de recorrido
while j < 630:
    j += 1
    
    #URL de extracción de datos
    url = 'https://www.summitpost.org/object_list.php?search_in=name_only&map_1=1&order_type=DESC&object_type=1&orderby=elevation&page='
    
    #Agregar variable de recorrido de páginas
    indice = str(j)
    url_completa = url + indice
    #print(url_completa)

    #Técnica para cambiar de forma aleatoria de User Agent para hacer el web scraping
    #Fuente: https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/
    user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    ]
    for i in range(1,4):
        user_agent = random.choice(user_agent_list)

    headers = {'User-Agent': user_agent}

    #Enviar requerimiento a URL
    response = requests.get(url_completa,headers=headers, timeout=15)

    #Extracción de datos
    data = BeautifulSoup(response.text, 'html.parser')

    #Mediante revisión del código HTML de la página objetivo, se identifica que los
    #datos de interés están en DIVs
    divs=data.find('div');

    #Recorrer las DIVs buscando las clases que contienen los datos de interés
    for row in divs.findAll('div',{"class": "custom-card-item"}):
        
        #Nombre de la montaña
        nombre = row.findAll('p',{"class": "cci-title"})
        #Limpieza de los datos
        nombre_limpio = remove_html_tags(str(nombre))
        nombre_limpio = nombre_limpio.replace("\n", "")
        nombre_limpio = nombre_limpio.replace("[", "")
        nombre_limpio = nombre_limpio.replace("]", "")
        nombre_limpio = '"' + nombre_limpio.strip() + '"'
        #print(nombre_limpio)

        #Altura de la montaña
        #Se identificaron casos en los cuales no está presente la etiqueta de altura
        #por lo que se les asigna valor de "ND" No Disponible para estos casos
        altura = row.findAll('span',{"class": "cci-elevation"}) or ("ND")

        #Limpieza de los datos
        if altura!="ND":
            altura_limpio = remove_html_tags(str(altura))
            partes_altura = altura_limpio.split(' ')
            altura_metros = partes_altura[5]
            altura_metros = altura_metros.replace("\n", "")
            altura_metros = altura_metros.replace("[", "")
            altura_metros = altura_metros.replace("]", "")
            altura = altura_metros
        #print(altura)
        
        #Puntaje asignado por el sitio a la montaña
        #calculado en función de visitas y votos
        #Detalle sobre signficado del puntaje (score): https://www.summitpost.org/faq
        puntaje = row.findAll('span',{"class": "cci-score"})
        #Limpieza de los datos
        puntaje_limpio = remove_html_tags(str(puntaje))
        puntaje_limpio = puntaje_limpio.replace("\n", "")
        puntaje_limpio = puntaje_limpio.replace("[", "")
        puntaje_limpio = puntaje_limpio.replace("]", "")
        puntaje_limpio = puntaje_limpio.replace("Score: ", "")
        #print(puntaje_limpio)
        
        #Cantidad de vistas o visitas a la página de la montaña
        vistas = row.findAll('span',{"class": "cci-hits"})
        #Limpieza de los datos
        vistas_limpio = remove_html_tags(str(vistas))
        vistas_limpio = vistas_limpio.replace("Views: ", "")
        vistas_limpio = vistas_limpio.replace("[", "")
        vistas_limpio = vistas_limpio.replace("]", "")
        #print(vistas_limpio)
        
        lugar = row.findAll('p',{"class": "cci-location"})
        #Limpieza de los datos
        lugar_limpio = remove_html_tags(str(lugar))
        lugar_limpio = lugar_limpio.replace("\n", "")
        lugar_limpio = lugar_limpio.replace("[", "")
        lugar_limpio = lugar_limpio.replace("]", "")
        lugar_limpio = ''.join(lugar_limpio.split())
        lugar_limpio = '"' + lugar_limpio.strip() + '"'
        #print(lugar_limpio)

        print("Página: ")
        print(j)
        print("\n")

        #Agregar línea en archivo CSV
        with open('montanasSummitPost.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([k, nombre_limpio, altura, puntaje_limpio, vistas_limpio, lugar_limpio])
            k += 1

print("Terminado =)")


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
1


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
2


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
3


Página: 
4


Página: 
4


Página: 
4


Página: 
4


Página: 
4



Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
27


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
28


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página: 
29


Página

Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
52


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
53


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página: 
54


Página

Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
77


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
78


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página: 
79


Página

Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
102


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
103


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
104


Página: 
1

...

Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
516


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
517


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
518


Página: 
5

Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
539


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
540


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
541


Página: 
5

Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
562


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
563


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
564


Página: 
5

Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
585


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
586


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
587


Página: 
5

Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
608


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
609


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
610


Página: 
6